<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Training-Model-Without-Outliers" data-toc-modified-id="Training-Model-Without-Outliers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Training Model Without Outliers</a></span></li><li><span><a href="#Training-Model-For-Outliers-Classification" data-toc-modified-id="Training-Model-For-Outliers-Classification-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training Model For Outliers Classification</a></span></li><li><span><a href="#Combining-Submission:" data-toc-modified-id="Combining-Submission:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Combining Submission:</a></span></li></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import warnings
import gc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss

### Training Model Without Outliers

In [3]:
train_df2 = feather.read_dataframe('train_df_alpha')
test_df2 = feather.read_dataframe('test_df_alpha')

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
for df in [train_df2, test_df2]:
    for f in ['purchase_date_max','purchase_date_min','purchase_date_max_old',\
                     'purchase_date_min_old', 'observation_date_old']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [5]:
cols_excluded = ['purchase_date_max', 'purchase_date_max_old', 'card_id', 'first_active_month',
                 'target','outliers','card_id_size', 'card_id_size_old', 
                 'purchase_date_min', 'purchase_date_min_old','first_active_monthYear',
                 'first_active_monthMonth',
                 'first_active_monthWeek',
                 'first_active_monthDay',
                 'first_active_monthDayofweek',
                 'first_active_monthDayofyear',
                 'first_active_monthIs_month_end',
                 'first_active_monthIs_month_start',
                 'first_active_monthIs_quarter_end',
                 'first_active_monthIs_quarter_start',
                 'first_active_monthIs_year_end',
                 'Black_Friday_2017_mean',
                 'amount_month_ratio_max',
                 'purchase_Month_mean_old',
                 'purchase_amount_total_max',
                 'first_active_monthIs_year_start']

In [6]:
cols_included = ['feature_1','feature_2','feature_3','transactions_count','subsector_id_nunique','merchant_id_nunique','merchant_category_id_nunique','purchase_Month_mean','purchase_Month_min','purchase_Month_max','purchase_Week_nunique','purchase_Week_mean','purchase_Week_min','purchase_Week_max','purchase_Dayofweek_mean','purchase_Dayofweek_min','purchase_Dayofweek_max','purchase_Day_nunique','purchase_Day_mean','purchase_Day_min','purchase_Day_max','purchase_Hour_nunique','purchase_Hour_mean','purchase_Hour_min','purchase_Hour_max','purchase_amount_sum','purchase_amount_max','purchase_amount_min','purchase_amount_mean','purchase_amount_var','purchase_amount_skew','installments_sum','installments_max','installments_mean','installments_var','installments_skew','month_lag_max','month_lag_min','month_lag_mean','month_lag_var','month_lag_skew','month_diff_mean','month_diff_var','month_diff_skew','purchased_on_weekend_mean','category_1_mean','category_2_mean','category_3_mean','card_id_count','price_mean','price_max','price_min','price_var','Christmas_Day_2017_mean','Children_day_2017_mean','Black_Friday_2017_mean','Mothers_Day_2018_mean','duration_mean','duration_min','duration_max','duration_var','duration_skew','amount_month_ratio_mean','amount_month_ratio_min','amount_month_ratio_max','amount_month_ratio_var','amount_month_ratio_skew','category_2_mean_mean','category_3_mean_mean','purchase_date_diff','purchase_date_average','purchase_date_uptonow','purchase_date_uptomin','transactions_count_old','subsector_id_nunique_old','merchant_id_nunique_old','merchant_category_id_nunique_old','purchase_Month_nunique','purchase_Month_mean_old','purchase_Month_min_old','purchase_Month_max_old','purchase_Week_nunique_old','purchase_Week_mean_old','purchase_Week_min_old','purchase_Week_max_old','purchase_Dayofweek_mean_old','purchase_Day_nunique_old','purchase_Day_mean_old','purchase_Day_min_old','purchase_Hour_nunique_old','purchase_Hour_mean_old','purchase_Hour_min_old','purchase_Hour_max_old','purchase_amount_sum_old','purchase_amount_max_old','purchase_amount_min_old','purchase_amount_mean_old','purchase_amount_var_old','purchase_amount_skew_old','installments_sum_old','installments_max_old','installments_mean_old','installments_var_old','installments_skew_old','month_lag_max_old','month_lag_min_old','month_lag_mean_old','month_lag_var_old','month_lag_skew_old','month_diff_max','month_diff_min','month_diff_mean_old','month_diff_var_old','month_diff_skew_old','authorized_flag_mean','purchased_on_weekend_mean_old','category_1_mean_old','category_2_mean_old','category_3_mean_old','card_id_count_old','price_sum','price_mean_old','price_max_old','price_min_old','price_var_old','Christmas_Day_2017_mean_old','Mothers_Day_2017_mean','fathers_day_2017_mean','Children_day_2017_mean_old','Valentine_Day_2017_mean','Black_Friday_2017_mean_old','Mothers_Day_2018_mean_old','duration_mean_old','duration_min_old','duration_max_old','duration_var_old','duration_skew_old','amount_month_ratio_mean_old','amount_month_ratio_min_old','amount_month_ratio_max_old','amount_month_ratio_var_old','amount_month_ratio_skew_old','category_2_mean_mean_old','category_3_mean_mean_old','purchase_date_diff_old','purchase_date_average_old','purchase_date_uptonow_old','purchase_date_uptomin_old','quarter','observed_elapsed_time','days_feature1','days_feature2','days_feature3','days_feature1_ratio','days_feature2_ratio','days_feature3_ratio','feature_sum','feature_mean','feature_max','feature_min','feature_var','card_id_total','card_id_count_total','card_id_count_ratio','purchase_amount_total','purchase_amount_total_mean','purchase_amount_total_max','purchase_amount_total_min','purchase_amount_sum_ratio','hist_first_buy','new_first_buy','hist_last_buy','new_last_buy','month_diff_ratio','installments_total','installments_ratio','price_total','CLV','CLV_old','CLV_ratio']

In [7]:
extra_cols = ['merchant_address_id_nunique', 'merchant_rating_nunique']

In [8]:
df_train_columns = [c for c in cols_included if c not in cols_excluded]

In [9]:
df_train_columns = df_train_columns + extra_cols

In [10]:
len(df_train_columns)

178

In [11]:
train_df2 = train_df2[train_df2['outliers'] == 0]

In [12]:
target = train_df2['target']

In [13]:
del train_df2['target']

In [14]:
len(df_train_columns)

178

In [15]:
gc.collect()

12

In [16]:
%%time
n_fold=5
param = {'objective':'regression',
         'num_leaves': 31,
         'min_data_in_leaf': 25,
         'max_depth': 7,
         'learning_rate': 0.01,
         'lambda_l1':0.13,
         "boosting": "gbdt",
         "feature_fraction":0.85,
         'bagging_freq':8,
         "bagging_fraction": 0.9 ,
         "metric": 'rmse',
         "verbosity": -1,
         "random_state": 2333}
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=4590)
oof = np.zeros(len(train_df2))
predictions = np.zeros(len(test_df2))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df2,train_df2['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df2.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df2.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train_df2.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df2[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

fold 0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.59443	valid_1's rmse: 1.62578
[200]	training's rmse: 1.56186	valid_1's rmse: 1.59798
[300]	training's rmse: 1.54754	valid_1's rmse: 1.58832
[400]	training's rmse: 1.53858	valid_1's rmse: 1.58359
[500]	training's rmse: 1.53166	valid_1's rmse: 1.58081
[600]	training's rmse: 1.52572	valid_1's rmse: 1.57891
[700]	training's rmse: 1.52048	valid_1's rmse: 1.57782
[800]	training's rmse: 1.51555	valid_1's rmse: 1.57696
[900]	training's rmse: 1.51108	valid_1's rmse: 1.57631
[1000]	training's rmse: 1.50674	valid_1's rmse: 1.57584
[1100]	training's rmse: 1.50253	valid_1's rmse: 1.57576
[1200]	training's rmse: 1.49858	valid_1's rmse: 1.57554
[1300]	training's rmse: 1.49482	valid_1's rmse: 1.57538
[1400]	training's rmse: 1.49115	valid_1's rmse: 1.57523
[1500]	training's rmse: 1.48751	valid_1's rmse: 1.57525
[1600]	training's rmse: 1.48386	valid_1's rmse: 1.57499
[1700]	training's rmse: 1.48031	valid_1's r

In [17]:
np.sqrt(mean_squared_error(oof, target))

1.553602833333153

1. 1.5545434532072333 - with existing params
2. 1.553654745445917 - with kernel params
3. 1.553602833333153 - latest with 178 cols

In [18]:
model_without_outliers = pd.DataFrame({"card_id":test_df2["card_id"].values})
model_without_outliers["target"] = predictions

### Training Model For Outliers Classification

In [19]:
train_df2 = feather.read_dataframe('train_df_alpha')
test_df2 = feather.read_dataframe('test_df_alpha')

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [20]:
for df in [train_df2, test_df2]:
    for f in ['purchase_date_max','purchase_date_min','purchase_date_max_old',\
                     'purchase_date_min_old', 'observation_date_old']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [21]:
cols_excluded = ['purchase_date_max', 'purchase_date_max_old', 'card_id', 'first_active_month',
                 'target','outliers','card_id_size', 'card_id_size_old', 
                 'purchase_date_min', 'purchase_date_min_old','first_active_monthYear',
                 'first_active_monthMonth',
                 'first_active_monthWeek',
                 'first_active_monthDay',
                 'first_active_monthDayofweek',
                 'first_active_monthDayofyear',
                 'first_active_monthIs_month_end',
                 'first_active_monthIs_month_start',
                 'first_active_monthIs_quarter_end',
                 'first_active_monthIs_quarter_start',
                 'first_active_monthIs_year_end',
                 'Black_Friday_2017_mean',
                 'amount_month_ratio_max',
                 'purchase_Month_mean_old',
                 'purchase_amount_total_max',
                 'first_active_monthIs_year_start']

In [22]:
cols_included = ['feature_1','feature_2','feature_3','transactions_count','subsector_id_nunique','merchant_id_nunique','merchant_category_id_nunique','purchase_Month_mean','purchase_Month_min','purchase_Month_max','purchase_Week_nunique','purchase_Week_mean','purchase_Week_min','purchase_Week_max','purchase_Dayofweek_mean','purchase_Dayofweek_min','purchase_Dayofweek_max','purchase_Day_nunique','purchase_Day_mean','purchase_Day_min','purchase_Day_max','purchase_Hour_nunique','purchase_Hour_mean','purchase_Hour_min','purchase_Hour_max','purchase_amount_sum','purchase_amount_max','purchase_amount_min','purchase_amount_mean','purchase_amount_var','purchase_amount_skew','installments_sum','installments_max','installments_mean','installments_var','installments_skew','month_lag_max','month_lag_min','month_lag_mean','month_lag_var','month_lag_skew','month_diff_mean','month_diff_var','month_diff_skew','purchased_on_weekend_mean','category_1_mean','category_2_mean','category_3_mean','card_id_count','price_mean','price_max','price_min','price_var','Christmas_Day_2017_mean','Children_day_2017_mean','Black_Friday_2017_mean','Mothers_Day_2018_mean','duration_mean','duration_min','duration_max','duration_var','duration_skew','amount_month_ratio_mean','amount_month_ratio_min','amount_month_ratio_max','amount_month_ratio_var','amount_month_ratio_skew','category_2_mean_mean','category_3_mean_mean','purchase_date_diff','purchase_date_average','purchase_date_uptonow','purchase_date_uptomin','transactions_count_old','subsector_id_nunique_old','merchant_id_nunique_old','merchant_category_id_nunique_old','purchase_Month_nunique','purchase_Month_mean_old','purchase_Month_min_old','purchase_Month_max_old','purchase_Week_nunique_old','purchase_Week_mean_old','purchase_Week_min_old','purchase_Week_max_old','purchase_Dayofweek_mean_old','purchase_Day_nunique_old','purchase_Day_mean_old','purchase_Day_min_old','purchase_Hour_nunique_old','purchase_Hour_mean_old','purchase_Hour_min_old','purchase_Hour_max_old','purchase_amount_sum_old','purchase_amount_max_old','purchase_amount_min_old','purchase_amount_mean_old','purchase_amount_var_old','purchase_amount_skew_old','installments_sum_old','installments_max_old','installments_mean_old','installments_var_old','installments_skew_old','month_lag_max_old','month_lag_min_old','month_lag_mean_old','month_lag_var_old','month_lag_skew_old','month_diff_max','month_diff_min','month_diff_mean_old','month_diff_var_old','month_diff_skew_old','authorized_flag_mean','purchased_on_weekend_mean_old','category_1_mean_old','category_2_mean_old','category_3_mean_old','card_id_count_old','price_sum','price_mean_old','price_max_old','price_min_old','price_var_old','Christmas_Day_2017_mean_old','Mothers_Day_2017_mean','fathers_day_2017_mean','Children_day_2017_mean_old','Valentine_Day_2017_mean','Black_Friday_2017_mean_old','Mothers_Day_2018_mean_old','duration_mean_old','duration_min_old','duration_max_old','duration_var_old','duration_skew_old','amount_month_ratio_mean_old','amount_month_ratio_min_old','amount_month_ratio_max_old','amount_month_ratio_var_old','amount_month_ratio_skew_old','category_2_mean_mean_old','category_3_mean_mean_old','purchase_date_diff_old','purchase_date_average_old','purchase_date_uptonow_old','purchase_date_uptomin_old','quarter','observed_elapsed_time','days_feature1','days_feature2','days_feature3','days_feature1_ratio','days_feature2_ratio','days_feature3_ratio','feature_sum','feature_mean','feature_max','feature_min','feature_var','card_id_total','card_id_count_total','card_id_count_ratio','purchase_amount_total','purchase_amount_total_mean','purchase_amount_total_max','purchase_amount_total_min','purchase_amount_sum_ratio','hist_first_buy','new_first_buy','hist_last_buy','new_last_buy','month_diff_ratio','installments_total','installments_ratio','price_total','CLV','CLV_old','CLV_ratio']

In [23]:
df_train_columns = [c for c in cols_included if c not in cols_excluded]

In [24]:
df_train_columns = df_train_columns + extra_cols

In [25]:
df_train = train_df2[df_train_columns]
target = train_df2['outliers']

In [26]:
col_types = df_train[df_train_columns].nunique().reset_index()

In [27]:
col_types.columns = ['f', 'uniq']

In [28]:
categorical_feats = col_types[col_types['uniq'] < 6]['f'].values

In [29]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333}

In [30]:
features = df_train_columns

In [31]:
categorical_feats = categorical_feats.tolist()

In [33]:
%%time
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(test_df2))
feature_importance_df = pd.DataFrame()

start = time.time()


for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df2[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(log_loss(target, oof)))

fold n°0


/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0437081	valid_1's binary_logloss: 0.0463519
[200]	training's binary_logloss: 0.0437029	valid_1's binary_logloss: 0.0463364
Early stopping, best iteration is:
[61]	training's binary_logloss: 0.0436896	valid_1's binary_logloss: 0.0462673
fold n°1


/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0437631	valid_1's binary_logloss: 0.0450172
[200]	training's binary_logloss: 0.0437399	valid_1's binary_logloss: 0.0449974
[300]	training's binary_logloss: 0.0437419	valid_1's binary_logloss: 0.0449875
[400]	training's binary_logloss: 0.0437386	valid_1's binary_logloss: 0.0449768
[500]	training's binary_logloss: 0.0437296	valid_1's binary_logloss: 0.0449539
[600]	training's binary_logloss: 0.0437311	valid_1's binary_logloss: 0.0449671
[700]	training's binary_logloss: 0.0437213	valid_1's binary_logloss: 0.0449651
Early stopping, best iteration is:
[503]	training's binary_logloss: 0.0437281	valid_1's binary_logloss: 0.0449496
fold n°2


...

Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0443527	valid_1's binary_logloss: 0.0434314
[200]	training's binary_logloss: 0.0443462	valid_1's binary_logloss: 0.0434375
[300]	training's binary_logloss: 0.0443533	valid_1's binary_logloss: 0.0434344
[400]	training's binary_logloss: 0.044338	valid_1's binary_logloss: 0.0434282
[500]	training's binary_logloss: 0.0443456	valid_1's binary_logloss: 0.0434261
Early stopping, best iteration is:
[395]	training's binary_logloss: 0.044335	valid_1's binary_logloss: 0.0434254
fold n°3


/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.042842	valid_1's binary_logloss: 0.0491988
[200]	training's binary_logloss: 0.0428534	valid_1's binary_logloss: 0.0491674
Early stopping, best iteration is:
[95]	training's binary_logloss: 0.0428333	valid_1's binary_logloss: 0.0492037
fold n°4


/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0439785	valid_1's binary_logloss: 0.0444379
[200]	training's binary_logloss: 0.0440146	valid_1's binary_logloss: 0.0444605
Early stopping, best iteration is:
[76]	training's binary_logloss: 0.043957	valid_1's binary_logloss: 0.0444204
CV score: 0.04565 
CPU times: user 5min 32s, sys: 4.97 s, total: 5min 37s
Wall time: 57.4 s


1. 0.09934738333325951 CV score 
2. Kernels score - 0.04492 - regression params
3. kernel params - 0.04556 

In [34]:
df_outlier_prob = pd.DataFrame({"card_id":test_df2["card_id"].values})
df_outlier_prob["target"] = predictions
df_outlier_prob.head()

,card_id,target
0,C_ID_0ab67a22ab,0.041347
1,C_ID_130fd0cbdd,0.001878
2,C_ID_b709037bc5,0.004944
3,C_ID_d27d835a9f,0.001878
4,C_ID_2b5e3df5c2,0.001932


In [35]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(target,oof)

In [36]:
fpr, tpr, thresholds

(array([0.     , 0.00001, 0.00002, ..., 0.99739, 0.99741, 1.     ]),
 array([0., 0., 0., ..., 1., 1., 1.]),
 array([1.48245, 0.48245, 0.43896, ..., 0.00163, 0.00163, 0.00162]))

In [37]:
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

In [38]:
optimal_threshold

0.011256291179551929

In [40]:
(df_outlier_prob['target'] > optimal_threshold).sum()

20474

### Combining Submission:

In [41]:
outlier_id = pd.DataFrame(df_outlier_prob.sort_values(by='target',
                                                      ascending = False).head(20474)['card_id'])

In [42]:
best_submission = pd.read_csv('./dl_xgb_submission_stacked_lgb-2019-02-26 22:45:24.597929.csv')

In [43]:
most_likely_liers = best_submission.merge(outlier_id,how='right')
most_likely_liers.head()

,card_id,target
0,C_ID_0ab67a22ab,-2.122841
1,C_ID_6d8dba8475,-0.849764
2,C_ID_7f1041e8e1,-6.443781
3,C_ID_22e4a47c72,-0.487886
4,C_ID_b54cfad8b2,-0.969509


In [56]:
# %%time
# for card_id in most_likely_liers['card_id']:
#     model_without_outliers.loc[model_without_outliers['card_id']==card_id,'target']\
#     = most_likely_liers.loc[most_likely_liers['card_id']==card_id,'target'].values

In [57]:
%%time
ix = best_submission["card_id"].isin(outlier_id["card_id"].values)
model_without_outliers.loc[ix, "target"] = best_submission[ix]["target"] 

CPU times: user 12 ms, sys: 1e+03 ns, total: 12 ms
Wall time: 11.6 ms


In [47]:
submission_name = pd.to_datetime("today")

In [48]:
filename = f'outlier_combining_submission-{submission_name}.csv'

In [49]:
model_without_outliers.to_csv(filename, index=False)

In [50]:
filename

'outlier_combining_submission-2019-02-26 23:20:23.680409.csv'

In [51]:
!kaggle competitions submit elo-merchant-category-recommendation -f 'outlier_combining_submission-2019-02-26 23:20:23.680409.csv' -m "combining model without outliers with threshold"

100%|#######################################| 4.23M/4.23M [00:22<00:00, 197kB/s]
Successfully submitted to Elo Merchant Category Recommendation

In [54]:
%%time
ix = best_submission["card_id"].isin(outlier_id["card_id"].values)
model_without_outliers.loc[ix, "target"] = best_submission[ix]["target"] 

CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 14.9 ms


In [55]:
model_without_outliers.tail()

,card_id,target
123618,C_ID_7a239d2eda,1.417893
123619,C_ID_75ace375ae,-0.828231
123620,C_ID_21d56d950c,1.114384
123621,C_ID_6c46fc5a9d,-2.949170
123622,C_ID_87e7979a5f,0.172589


In [52]:
model_without_outliers.tail()

,card_id,target
123618,C_ID_7a239d2eda,1.417893
123619,C_ID_75ace375ae,-0.828231
123620,C_ID_21d56d950c,1.114384
123621,C_ID_6c46fc5a9d,-2.949170
123622,C_ID_87e7979a5f,0.172589
